In [1]:
import csv
import os
import requests

# 1. Nekaj pomožnih orodij za pridobivanje podatkov s spleta

In [2]:
# definirajte URL glavne strani bolhe za oglase z mačkami
cats_frontpage_url = 'https://www.bolha.com/macke'
# mapa, v katero bomo shranili podatke
cat_directory = 'podatki'
# ime datoteke, v katero bomo shranili glavno stran
frontpage_filename = 'glavna.html'
# ime CSV datoteke, v katero bomo shranili podatke
csv_filename = 'macke.csv'

In [3]:
print(csv_filename)

macke.csv


In [4]:
import traceback  # za popolno poročilo o napaki


def download_url_to_string(url):
    """Funkcija kot argument sprejme niz in poskusi vrniti vsebino te spletne
    strani kot niz. V primeru, da med izvajanje pride do napake vrne None.
    """
    try:
        # del kode, ki morda sproži napako
        page_content = requests.get(url)
        if page_content.status_code == 200:
            return page_content.text
        else:
            raise ValueError(f"Čudna koda: {page_content.status_code}")
    except Exception:
        # koda, ki se izvede pri napaki
        # dovolj je če izpišemo opozorilo in prekinemo izvajanje funkcije
        print(f"Prišlo je do spodnje napake:\n{traceback.format_exc()}")


def save_string_to_file(text, directory, filename):
    """Funkcija zapiše vrednost parametra "text" v novo ustvarjeno datoteko
    locirano v "directory"/"filename", ali povozi obstoječo. V primeru, da je
    niz "directory" prazen datoteko ustvari v trenutni mapi.
    """
    os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, filename)
    with open(path, 'w', encoding='utf-8') as file_out:
        file_out.write(text)
    return None

Definirajte funkcijo (s pomočjo gornjih dveh), ki prenese glavno stran in jo shrani v datoteko.

V ločeni celici jo nato pokličite in shranite naslovno stran oglasov za mačke. Če želite še več oglasov, prilagodite url in mu dodajte npr. `?page=3`.

In [5]:
def save_frontpage(page, directory, filename):
    """Funkcija shrani vsebino spletne strani na naslovu "page" v datoteko
    "directory"/"filename"."""
    html_strani = download_url_to_string(page)
    save_string_to_file(html_strani, directory, filename)


In [9]:
save_frontpage(cats_frontpage_url, cat_directory, frontpage_filename)

# 2. Po pridobitvi podatkov jih želimo obdelati

Definirajmo nekaj funkcij - vsaka bo opravila en kos dela.

In [6]:
def read_file_to_string(directory, filename):
    """Funkcija vrne celotno vsebino datoteke "directory"/"filename" kot niz."""
    path = os.path.join(directory, filename)
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

Definirajte funkcijo, ki sprejme niz, ki predstavlja vsebino spletne strani,
in ga razdeli na dele, kjer vsak del predstavlja en oglas. To storite s
pomočjo regularnih izrazov, ki označujejo začetek in konec posameznega
oglasa. Funkcija naj vrne seznam nizov.

**Komentar**: Na vajah se je izkazalo, da vzorec

```
vzorec = "<article.+?</article>"
```

ni popoln, saj nam da še nekatere druge oglase poleg mačk, ki jih želimo, kar je kasneje privedlo do napak pri iskanju cene ipd.

Popravimo ga tako, da preverimo, da se povezava, ki je prisotna v oglasu, res nanaša na mačke.


In [7]:
import re


def page_to_ads(page_content):
    """Funkcija poišče posamezne oglase, ki se nahajajo v spletni strani in
    vrne seznam oglasov."""
    vzorec = '<article.+?href="/macke.+?</article>'
    return re.findall(vzorec, page_content, flags=re.DOTALL)

In [21]:
print(os.listdir())
vsebina = read_file_to_string(cat_directory, frontpage_filename)
oglasi = page_to_ads(vsebina)
print(len(oglasi))

['macke.ipynb', 'macke.py', 'orig.py', 'podatki', 'zajemanjePodatkov', 'zajemanjePodatkov.zip']
26


Definirajte funkcijo, ki sprejme niz, ki predstavlja oglas, in izlušči
podatke o imenu, lokaciji, datumu objave in ceni v oglasu.


In [19]:
def get_dict_from_ad_block(block):
    """Funkcija iz niza za posamezen oglasni blok izlušči podatke o imenu, ceni
    in opisu ter vrne slovar, ki vsebuje ustrezne podatke."""
    vzorec_ime = '<h3 class="entity-title">.+?>(.+?)<'
    vzorec_cena = r'<strong class="price price--hrk">.*?(\d+|Cena po dogovoru)'
    vzorec_datum = r'pubdate="pubdate">([0-9.]+)<'
    try:
        ime = re.search(vzorec_ime, block).group(1)
        cena = re.search(vzorec_cena, block, flags=re.DOTALL).group(1)
        datum = re.search(vzorec_datum, block).group(1)
    except AttributeError:
        print(f"Nepopolni vzorci pri (čudnem?) oglasu\n{block}")
        raise
    return {"ime": ime, "cena": cena, "datum": datum}

In [22]:
for oglas in oglasi:
    print(get_dict_from_ad_block(oglas))

{'ime': 'Britanka samička BLH WCF Pedigree', 'cena': '750', 'datum': '16.05.2023.'}
{'ime': 'Britanka - britanke', 'cena': '500', 'datum': '17.05.2023.'}
{'ime': 'Ragdoll mladički', 'cena': 'Cena po dogovoru', 'datum': '17.05.2023.'}
{'ime': 'SOFI', 'cena': 'Cena po dogovoru', 'datum': '17.05.2023.'}
{'ime': 'VLADKO', 'cena': 'Cena po dogovoru', 'datum': '17.05.2023.'}
{'ime': 'ALFI (rezerviran)', 'cena': 'Cena po dogovoru', 'datum': '17.05.2023.'}
{'ime': 'Britanec za parjenje', 'cena': 'Cena po dogovoru', 'datum': '16.05.2023.'}
{'ime': 'Mala črna igriva in cartljiva mucka išče novi dom', 'cena': '0', 'datum': '16.05.2023.'}
{'ime': 'Britanka samička BLH WCF Pedigree', 'cena': '750', 'datum': '16.05.2023.'}
{'ime': 'prodam britanko', 'cena': '300', 'datum': '16.05.2023.'}
{'ime': 'Britanke modre', 'cena': '500', 'datum': '16.05.2023.'}
{'ime': 'BENGALKA - MLADIČKI S FIFe RODOVNIKOM', 'cena': '1', 'datum': '16.05.2023.'}
{'ime': 'BRITANKA - MLADIČKI čistokrvni z rodovniki', 'cena': 'C

Definirajte funkcijo, ki sprejme ime in lokacijo datoteke, ki vsebuje
besedilo spletne strani, in vrne seznam slovarjev, ki vsebujejo podatke o
vseh oglasih strani.

In [23]:
def ads_from_file(filename, directory):
    """Funkcija prebere podatke v datoteki "directory"/"filename" in jih
    pretvori (razčleni) v pripadajoč seznam slovarjev za vsak oglas posebej."""
    vsebina = read_file_to_string(directory, filename)
    oglasi = page_to_ads(vsebina)
    slovarji = []
    for oglas in oglasi:
        slovarji.append(get_dict_from_ad_block(oglas))
    return slovarji

In [24]:
ads_from_file(frontpage_filename, cat_directory)

[{'ime': 'Britanka samička BLH WCF Pedigree',
  'cena': '750',
  'datum': '16.05.2023.'},
 {'ime': 'Britanka - britanke', 'cena': '500', 'datum': '17.05.2023.'},
 {'ime': 'Ragdoll mladički',
  'cena': 'Cena po dogovoru',
  'datum': '17.05.2023.'},
 {'ime': 'SOFI', 'cena': 'Cena po dogovoru', 'datum': '17.05.2023.'},
 {'ime': 'VLADKO', 'cena': 'Cena po dogovoru', 'datum': '17.05.2023.'},
 {'ime': 'ALFI (rezerviran)',
  'cena': 'Cena po dogovoru',
  'datum': '17.05.2023.'},
 {'ime': 'Britanec za parjenje',
  'cena': 'Cena po dogovoru',
  'datum': '16.05.2023.'},
 {'ime': 'Mala črna igriva in cartljiva mucka išče novi dom',
  'cena': '0',
  'datum': '16.05.2023.'},
 {'ime': 'Britanka samička BLH WCF Pedigree',
  'cena': '750',
  'datum': '16.05.2023.'},
 {'ime': 'prodam britanko', 'cena': '300', 'datum': '16.05.2023.'},
 {'ime': 'Britanke modre', 'cena': '500', 'datum': '16.05.2023.'},
 {'ime': 'BENGALKA - MLADIČKI S FIFe RODOVNIKOM',
  'cena': '1',
  'datum': '16.05.2023.'},
 {'ime': 'BR

# 3. Shranimo obdelane podatke

In [28]:
def write_csv(fieldnames, rows, directory, filename):
    """
    Funkcija v csv datoteko podano s parametroma "directory"/"filename" zapiše
    vrednosti v parametru "rows" pripadajoče ključem podanim v "fieldnames"
    """
    os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, filename)
    # ko odpremo datoteko, podamo neobevzni argument newline in ga nastavimo na prazen niz,
    # sicer bomo na windowsih imeli grd csv, kjer bo vsaki dejanski vrstici sledila prazna
    with open(path, 'w', encoding='utf-8', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)
    return

Definirajte funkcijo, ki sprejme neprazen seznam slovarjev, ki predstavljajo
podatke iz oglasa mačke, in zapiše vse podatke v csv datoteko. Imena za
stolpce [fieldnames] pridobite iz slovarjev.

In [29]:
def write_cat_ads_to_csv(ads, directory, filename):
    """Funkcija vse podatke iz parametra "ads" zapiše v csv datoteko podano s
    parametroma "directory"/"filename". Funkcija predpostavi, da so ključi vseh
    slovarjev parametra ads enaki in je seznam ads neprazen."""
    # Stavek assert preveri da zahteva velja
    # Če drži se program normalno izvaja, drugače pa sproži napako
    # Prednost je v tem, da ga lahko pod določenimi pogoji izklopimo v
    # produkcijskem okolju
    assert ads and (all(slovar.keys() == ads[0].keys() for slovar in ads))
    imena_stolpcev = sorted(ads[0])
    write_csv(imena_stolpcev, ads, directory, filename)
    

In [30]:
vsi_slovarji = ads_from_file(frontpage_filename, cat_directory)
write_cat_ads_to_csv(vsi_slovarji, "obdelani_podatki", csv_filename)

# 4. Vse skupaj

Ko imamo enkrat vse pripravljeno, je koristno, da vse korake združimo v eno samo funkcijo, ki z enim klikom opravi vse delo.


In [31]:
def main(redownload=True, reparse=True):
    """Funkcija izvede celoten del pridobivanja podatkov:
    1. Oglase prenese iz bolhe
    2. Lokalno html datoteko pretvori v lepšo predstavitev podatkov
    3. Podatke shrani v csv datoteko
    """
    # 1. Najprej v lokalno datoteko shranimo glavno stran
    pot_html = os.path.join(cat_directory, frontpage_filename)
    if redownload or not os.path.exists(pot_html):
        # če zahtevamo ponovno nalaganje ali pa html_datoteka
        # še ne obstaja, jo naložimo s spleta
        save_frontpage(cats_frontpage_url, cat_directory, frontpage_filename)
    else:
        print(f"Datoteka {pot_html} že obstaja")
    csv_mapa = "obdelani_podatki"
    pot_csv = os.path.join(csv_mapa, csv_filename)
    # 2. in 3.: obdelava html-ja in shranjevanje v csv
    if reparse or not os.path.exists(pot_csv):
        # če zahtevamo ponovno obdelavo ali pa csv_datoteka
        # še ne obstaja, jo ustvarimo (ponovno)
        vsi_slovarji = ads_from_file(frontpage_filename, cat_directory)
        write_cat_ads_to_csv(vsi_slovarji, "obdelani_podatki", csv_filename)
    else:
        print(f"Datoteka {pot_csv} že obstaja")



In [32]:
main(False, False)

Datoteka podatki\glavna.html že obstaja
Datoteka obdelani_podatki\macke.csv že obstaja
